# Finnhub

In [ ]:
import requests
import pandas as pd
import datetime
import os
from dotenv import load_dotenv
import yfinance as yf
import time

# .env 설정 로드
load_dotenv()
FINHUB_API_KEY = os.getenv("finhub")  # 변수명은 'finnhub' 아닌 'finhub'

# API 호출 제한 (60 calls/minute)
API_CALLS_PER_MINUTE = 60
DELAY_BETWEEN_CALLS = 60.0 / API_CALLS_PER_MINUTE  # 1초 간격


def safe_datetime_conversion(timestamp):
    """
    타임스탬프를 안전하게 datetime으로 변환합니다.
    Out of bounds nanosecond timestamp 오류를 방지합니다.
    """
    if not timestamp or timestamp == 0:
        return None
    
    try:
        # 유닉스 타임스탬프 범위 확인 (1970-01-01 이후)
        if timestamp < 0:
            return None
            
        # 너무 큰 값 확인 (2262년 이후는 pandas에서 처리 불가)
        if timestamp > 9223372036:  # 2262-04-11 정도
            return None
            
        # 정상적인 변환 시도
        return pd.to_datetime(timestamp, unit='s')
        
    except (ValueError, OutOfBoundsDatetime, OverflowError):
        # 변환 실패 시 None 반환
        return None
    except Exception:
        # 기타 예외 시 None 반환
        return None


# Finnhub에서 회사 뉴스 수집 (최적화된 버전 - 대량 수집)
def fetch_finnhub_news_extended(symbol: str = "GOOGL", start_date: str = "2025-06-14", days_per_request: int = 30) -> pd.DataFrame:
    """
    하나의 심볼에 대해 최대한 많은 뉴스를 수집합니다.
    날짜 구간을 나누어서 여러 번 API 호출하여 더 많은 기사를 수집합니다.
    
    Args:
        symbol: 주식 심볼 (예: "AAPL")
        start_date: 시작 날짜 (YYYY-MM-DD)
        days_per_request: 한 번의 API 호출당 수집할 일수 (기본: 30일)
    """
    if not FINHUB_API_KEY:
        print("[ERROR] Finnhub API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")
        return pd.DataFrame()

    url = "https://finnhub.io/api/v1/company-news"
    
    try:
        start_date_obj = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
        today = datetime.date.today()
        
        print(f"[INFO] 요청 시작 날짜: {start_date_obj}")
        print(f"[INFO] 현재 날짜: {today}")
        
        # Free Tier 제한을 넘어서 최대한 많이 수집 시도
        two_years_ago = today - datetime.timedelta(days=730)  # 2년 전
        three_years_ago = today - datetime.timedelta(days=1095)  # 3년 전
        
        print(f"[INFO] 🚀 Free Tier 제한 돌파 시도: 2-3년간 뉴스 수집을 시도합니다!")
        print(f"[INFO] 2년 전 날짜: {two_years_ago}")
        print(f"[INFO] 3년 전 날짜: {three_years_ago}")
        
        # 3년 전부터 현재까지로 시도 (API가 어디까지 허용하는지 테스트)
        actual_start = three_years_ago
        actual_end = today
        
        print(f"[INFO] {symbol} 뉴스 대량 수집 시작 (제한 돌파 시도)")
        print(f"[INFO] 실제 수집 기간: {actual_start.isoformat()} ~ {actual_end.isoformat()}")
        print(f"[INFO] 수집 기간: {(actual_end - actual_start).days}일 (약 3년)")
        print(f"[INFO] 예상 API 호출 횟수: {(actual_end - actual_start).days // days_per_request + 1}회")
        print(f"[WARNING] API가 제한할 수 있습니다. 테스트 중...")
        
    except ValueError:
        print(f"[ERROR] 잘못된 날짜 형식: {start_date}. YYYY-MM-DD 형식을 사용하세요.")
        return pd.DataFrame()

    all_articles = []
    current_date = actual_start
    request_count = 0
    
    while current_date < actual_end:
        # 각 요청의 종료 날짜 계산
        period_end = min(current_date + datetime.timedelta(days=days_per_request), actual_end)
        
        params = {
            "symbol": symbol,
            "from": current_date.isoformat(),
            "to": period_end.isoformat(),
            "token": FINHUB_API_KEY
        }
        
        try:
            request_count += 1
            print(f"[INFO] API 호출 {request_count}: {current_date.isoformat()} ~ {period_end.isoformat()}")
            
            # API 호출 제한을 위한 딜레이
            time.sleep(DELAY_BETWEEN_CALLS)
            
            res = requests.get(url, params=params, timeout=30)
            
            if res.status_code == 429:
                print(f"[WARNING] API 호출 제한 도달. 더 긴 대기 후 재시도...")
                time.sleep(10)
                res = requests.get(url, params=params, timeout=30)
            
            if res.status_code == 403:
                print(f"[ERROR] API 접근 거부 (기간: {current_date} ~ {period_end}): Free Tier 제한 도달 가능성")
                print(f"[INFO] 현재까지 수집된 기사: {len(all_articles)}개")
                break
            
            if res.status_code != 200:
                print(f"[WARNING] API 요청 실패 (기간: {current_date} ~ {period_end}): HTTP {res.status_code}")
                print(f"[INFO] 응답 내용: {res.text[:200]}...")
                
                # 429 (Too Many Requests)가 아니라면 계속 진행
                if res.status_code != 429:
                    current_date = period_end + datetime.timedelta(days=1)
                    continue
                else:
                    print(f"[INFO] 호출 제한으로 인한 대기...")
                    time.sleep(30)
                    continue

            data = res.json()
            
            if not isinstance(data, list):
                print(f"[WARNING] 예상과 다른 응답 형식 (기간: {current_date} ~ {period_end})")
                if isinstance(data, dict) and 'error' in data:
                    print(f"[ERROR] API 오류: {data['error']}")
                    if 'limit' in data['error'].lower():
                        print(f"[INFO] Free Tier 제한 도달. 현재까지 수집: {len(all_articles)}개")
                        break
                current_date = period_end + datetime.timedelta(days=1)
                continue

            # 해당 기간의 뉴스 수집
            period_articles = []
            for item in data:
                # 안전한 datetime 변환 사용
                pub_date = safe_datetime_conversion(item.get("datetime"))
                
                article = {
                    "id": item.get("id"),
                    "title": item.get("headline", ""),
                    "summary": item.get("summary", ""),
                    "link": item.get("url", ""),
                    "publisher": item.get("publisher", ""),
                    "category": item.get("category", ""),
                    "pubDate": pub_date,
                    "image": item.get("image", ""),
                    "related": item.get("related", ""),
                    "source": item.get("source", ""),
                    "collection_period": f"{current_date.isoformat()}_{period_end.isoformat()}"
                }
                period_articles.append(article)
            
            all_articles.extend(period_articles)
            print(f"[SUCCESS] 기간별 수집: {len(period_articles)}개 기사 (총 {len(all_articles)}개)")
            
        except Exception as e:
            print(f"[ERROR] API 호출 오류 (기간: {current_date} ~ {period_end}): {e}")
            if "limit" in str(e).lower() or "403" in str(e):
                print(f"[INFO] Free Tier 제한 도달 가능성. 현재까지 수집: {len(all_articles)}개")
                break
        
        # 다음 기간으로 이동
        current_date = period_end + datetime.timedelta(days=1)
        
        # API 호출 제한 방지를 위한 추가 대기
        if request_count % 10 == 0:  # 10번 호출마다 추가 대기
            print(f"[INFO] API 제한 방지를 위한 대기... (현재까지 {len(all_articles)}개 수집)")
            time.sleep(2)

    # 전체 결과 처리
    if all_articles:
        df = pd.DataFrame(all_articles)
        
        # 중복 제거 (ID, 제목, 링크 기준)
        before_dedup = len(df)
        df = df.drop_duplicates(subset=['id', 'title', 'link'])
        after_dedup = len(df)
        
        if before_dedup != after_dedup:
            print(f"[INFO] 중복 기사 제거: {before_dedup - after_dedup}개")
        
        # 날짜순 정렬 (최신순) - None 값 처리
        df = df.sort_values('pubDate', ascending=False, na_position='last')
        
        print(f"[SUCCESS] {symbol} 총 {len(df)}개의 뉴스 기사 수집 완료!")
        print(f"[INFO] 총 API 호출 횟수: {request_count}회")
        
        return df
    else:
        print(f"[WARNING] {symbol}에 대한 뉴스를 찾을 수 없습니다.")
        return pd.DataFrame()


# yfinance 주가 데이터
def fetch_stock_data(ticker_symbol: str, period: str = '1y', interval: str = '1d') -> pd.DataFrame:
    try:
        df = yf.download(
            tickers=ticker_symbol,
            period=period,
            interval=interval,
            auto_adjust=False,
            progress=False
        )
        if df.empty:
            return pd.DataFrame()
        df = df.reset_index()
        df = df.rename(columns={'Adj Close': 'Adj_Close'})
        return df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']]
    except Exception as e:
        print(f"[ERROR] 주식 데이터 오류: {e}")
        return pd.DataFrame()


if __name__ == "__main__":
    # 대량 뉴스 수집할 단일 심볼 설정
    target_symbol = "GOOGL"  # 원하는 심볼로 변경 가능 (예: MSFT, GOOGL, AMZN, TSLA, META, NVDA 등)
    start_date = "2025-06-13"  # 참고용 (실제로는 1년 전부터 자동 수집)
    days_per_request = 7  # 한 번의 API 호출당 7일씩 수집 (더 많은 API 호출로 최대 수집)
    
    print(f"{target_symbol} 뉴스 대량 수집 시작!")
    print(f"참고 날짜: {start_date} (실제로는 3년 전부터 수집 시도)")
    print(f"수집 방식: {days_per_request}일씩 구간별 수집")
    print(f"API 제한: {API_CALLS_PER_MINUTE}회/분")
    print(f"목표:Free Tier 제한 돌파 시도 - 최대 3년간 뉴스 수집!")
    print(f"주의: API가 제한을 걸 수 있으니 실험적 수집입니다.")
    print("="*60)
    
    # 확장된 뉴스 수집 함수 사용
    df_extended_news = fetch_finnhub_news_extended(
        symbol=target_symbol, 
        start_date=start_date,
        days_per_request=days_per_request
    )
    
    if not df_extended_news.empty:
        print(f"\n🎉 {target_symbol} 뉴스 수집 완료!")
        print(f"📰 총 수집 기사 수: {len(df_extended_news)}개")
        
        # 날짜별 기사 분포 분석 (유효한 날짜만)
        if 'pubDate' in df_extended_news.columns:
            valid_dates = df_extended_news[df_extended_news['pubDate'].notna()].copy()
            if not valid_dates.empty:
                valid_dates['date_only'] = valid_dates['pubDate'].dt.date
                date_counts = valid_dates['date_only'].value_counts().sort_index()
                print(f"📈 수집 기간: {date_counts.index.min()} ~ {date_counts.index.max()}")
                print(f"📊 평균 일일 기사 수: {date_counts.mean():.1f}개")
                print(f"📅 유효한 날짜 기사: {len(valid_dates)}개 / 전체 {len(df_extended_news)}개")
        
        # 최신 기사 10개 미리보기
        print(f"\n📰 최신 뉴스 미리보기 (상위 10개)")
        print("-" * 80)
        for i, row in df_extended_news.head(10).iterrows():
            pub_date = row['pubDate'].strftime('%Y-%m-%d %H:%M') if pd.notna(row['pubDate']) else 'N/A'
            title = row['title'][:60] + "..." if len(row['title']) > 60 else row['title']
            publisher = row['publisher'] if row['publisher'] else row['source']
            print(f"{i+1:2d}. [{publisher}] {title}")
            print(f"    📅 {pub_date} | 🔗 {row['link'][:50]}...")
            print()
        
        # CSV 파일로 저장
        today = datetime.date.today().isoformat()
        filename = f"{target_symbol}_extended_news_{today}.csv"
        df_extended_news.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"💾 파일 저장 완료: {filename}")
        
        # 발행처별 통계
        print(f"\n📊 발행처별 기사 수 통계 (상위 10개)")
        print("-" * 40)
        publisher_col = 'publisher' if df_extended_news['publisher'].notna().any() else 'source'
        publisher_counts = df_extended_news[publisher_col].value_counts()
        for publisher, count in publisher_counts.head(10).items():
            if publisher:  # 빈 값이 아닌 경우만
                print(f"{publisher:25s}: {count:3d}개")
        
        # 카테고리별 통계 (있는 경우)
        if 'category' in df_extended_news.columns and df_extended_news['category'].notna().any():
            print(f"\n🏷️  카테고리별 기사 수")
            print("-" * 30)
            category_counts = df_extended_news['category'].value_counts()
            for category, count in category_counts.items():
                if category:
                    print(f"{category:20s}: {count:3d}개")
                    
    else:
        print(f"❌ {target_symbol} 뉴스 수집에 실패했습니다.")
        print("🔧 가능한 해결 방법:")
        print("   1. API 키 확인 (.env 파일의 'finhub' 변수)")
        print("   2. 인터넷 연결 확인")  
        print("   3. 심볼명 확인 (미국 상장 기업만 지원)")
        print("   4. 날짜 범위 조정")
    
    print("\n" + "="*60)
    print("📋 수집 완료 요약")
    print("="*60)
    print(f"🏢 대상 기업: {target_symbol}")
    print(f"📅 수집 날짜: {start_date}")
    print(f"📊 총 기사 수: {len(df_extended_news) if not df_extended_news.empty else 0}개")
    print(f"💾 저장 파일: {filename if not df_extended_news.empty else 'N/A'}")


# Y finance

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
import numpy as np
warnings.filterwarnings('ignore')

def get_hourly_stock_data(ticker, days=365, save_to_csv=True):
    """
    티커를 입력받아 최근 N일간의 1시간 간격 주식 데이터를 가져오는 함수
    
    Parameters:
    ticker (str): 주식 티커 심볼 (예: 'AAPL', 'TSLA', 'AMZN')
    days (int): 수집할 일수 (최대 730일, yfinance 제약)
    save_to_csv (bool): CSV 파일로 저장할지 여부
    
    Returns:
    pandas.DataFrame: 1시간 간격 주식 데이터
    """
    
    try:
        # yfinance 1시간 간격 제약사항 확인
        if days > 730:
            print(f"⚠️ yfinance 1시간 간격 데이터는 최대 730일까지만 지원됩니다.")
            print(f"요청한 {days}일 → 730일로 조정합니다.")
            days = 730
        
        # 날짜 설정 (현재 날짜 기준)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        print(f"📊 {ticker} 주식 데이터 수집 중...")
        print(f"기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')} ({days}일)")
        print(f"간격: 1시간")
        
        # yfinance로 데이터 수집 (24시간 데이터 포함)
        stock_data = yf.download(
            ticker,
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d'),
            interval='1h',
            prepost=True,  # 시장 외 시간 데이터 포함
            progress=False
        )
        
        if stock_data.empty:
            print(f"❌ {ticker}에 대한 데이터를 찾을 수 없습니다.")
            return None
        
        # 인덱스를 컬럼으로 변환
        stock_data = stock_data.reset_index()
        
        # 컬럼명 확인 및 정리
        print(f"🔍 원본 컬럼명: {list(stock_data.columns)}")
        
        # 인덱스 컬럼명 통일 (Datetime으로)
        if 'Date' in stock_data.columns:
            stock_data = stock_data.rename(columns={'Date': 'Datetime'})
        elif stock_data.columns[0] not in ['Datetime', 'Date']:
            # 첫 번째 컬럼이 시간 데이터인 경우
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # 멀티레벨 컬럼인 경우 처리
        if isinstance(stock_data.columns, pd.MultiIndex):
            new_columns = []
            for col in stock_data.columns:
                if isinstance(col, tuple):
                    if col[0] == 'Datetime' or 'Date' in str(col[0]):
                        new_columns.append('Datetime')
                    else:
                        new_columns.append(col[0])
                else:
                    new_columns.append(col)
            stock_data.columns = new_columns
        
        # 기본 정보 출력
        print(f"✅ 데이터 수집 완료!")
        print(f"총 데이터 포인트: {len(stock_data):,}개")
        print(f"정리된 컬럼명: {list(stock_data.columns)}")
        
        # Datetime 컬럼 확인
        if 'Datetime' in stock_data.columns:
            print(f"데이터 기간: {stock_data['Datetime'].min()} ~ {stock_data['Datetime'].max()}")
        else:
            print(f"⚠️ Datetime 컬럼을 찾을 수 없습니다. 첫 번째 컬럼 사용: {stock_data.columns[0]}")
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # 기본 통계
        print(f"\n📈 기본 통계:")
        print(f"시작 가격: ${stock_data['Open'].iloc[0]:.2f}")
        print(f"종료 가격: ${stock_data['Close'].iloc[-1]:.2f}")
        print(f"최고가: ${stock_data['High'].max():.2f}")
        print(f"최저가: ${stock_data['Low'].min():.2f}")
        print(f"평균 거래량: {stock_data['Volume'].mean():,.0f}")
        
        # 시간을 정시로 조정 (예: 13:30 -> 13:00)
        stock_data = adjust_time_to_hour(stock_data)
        
        # 추가 특성 계산
        stock_data = add_technical_features(stock_data)
        
        # CSV 저장
        if save_to_csv:
            filename = f"{ticker}_1hour_data_{days}days.csv"
            stock_data.to_csv(filename, index=False)
            print(f"💾 데이터가 '{filename}'에 저장되었습니다.")
        
        return stock_data
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print(f"오류 상세: {type(e).__name__}")
        return None

def get_30min_stock_data(ticker, days=60, save_to_csv=True):
    """
    티커를 입력받아 최근 N일간의 30분 간격 주식 데이터를 가져오는 함수
    
    Parameters:
    ticker (str): 주식 티커 심볼 (예: 'AAPL', 'TSLA', 'AMZN')
    days (int): 수집할 일수 (최대 60일, yfinance 제약)
    save_to_csv (bool): CSV 파일로 저장할지 여부
    
    Returns:
    pandas.DataFrame: 30분 간격 주식 데이터
    """
    
    try:
        # yfinance 30분 간격 제약사항 확인
        if days > 60:
            print(f"⚠️ yfinance 30분 간격 데이터는 최대 60일까지만 지원됩니다.")
            print(f"요청한 {days}일 → 60일로 조정합니다.")
            days = 60
        
        # 날짜 설정 (현재 날짜 기준)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        print(f"📊 {ticker} 주식 데이터 수집 중...")
        print(f"기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')} ({days}일)")
        print(f"간격: 30분")
        
        # yfinance로 데이터 수집 (24시간 데이터 포함)
        stock_data = yf.download(
            ticker,
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d'),
            interval='30m',
            prepost=True,  # 시장 외 시간 데이터 포함
            progress=False
        )
        
        if stock_data.empty:
            print(f"❌ {ticker}에 대한 데이터를 찾을 수 없습니다.")
            return None
        
        # 인덱스를 컬럼으로 변환
        stock_data = stock_data.reset_index()
        
        # 컬럼명 확인 및 정리
        print(f"🔍 원본 컬럼명: {list(stock_data.columns)}")
        
        # 인덱스 컬럼명 통일 (Datetime으로)
        if 'Date' in stock_data.columns:
            stock_data = stock_data.rename(columns={'Date': 'Datetime'})
        elif stock_data.columns[0] not in ['Datetime', 'Date']:
            # 첫 번째 컬럼이 시간 데이터인 경우
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # 멀티레벨 컬럼인 경우 처리
        if isinstance(stock_data.columns, pd.MultiIndex):
            new_columns = []
            for col in stock_data.columns:
                if isinstance(col, tuple):
                    if col[0] == 'Datetime' or 'Date' in str(col[0]):
                        new_columns.append('Datetime')
                    else:
                        new_columns.append(col[0])
                else:
                    new_columns.append(col)
            stock_data.columns = new_columns
        
        # 기본 정보 출력
        print(f"✅ 데이터 수집 완료!")
        print(f"총 데이터 포인트: {len(stock_data):,}개")
        print(f"정리된 컬럼명: {list(stock_data.columns)}")
        
        # Datetime 컬럼 확인
        if 'Datetime' in stock_data.columns:
            print(f"데이터 기간: {stock_data['Datetime'].min()} ~ {stock_data['Datetime'].max()}")
        else:
            print(f"⚠️ Datetime 컬럼을 찾을 수 없습니다. 첫 번째 컬럼 사용: {stock_data.columns[0]}")
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # 기본 통계
        print(f"\n📈 기본 통계:")
        print(f"시작 가격: ${stock_data['Open'].iloc[0]:.2f}")
        print(f"종료 가격: ${stock_data['Close'].iloc[-1]:.2f}")
        print(f"최고가: ${stock_data['High'].max():.2f}")
        print(f"최저가: ${stock_data['Low'].min():.2f}")
        print(f"평균 거래량: {stock_data['Volume'].mean():,.0f}")
        
        # 시간을 정시로 조정 (예: 13:30 -> 13:00)
        stock_data = adjust_time_to_hour(stock_data)
        
        # 추가 특성 계산
        stock_data = add_technical_features(stock_data)
        
        # CSV 저장
        if save_to_csv:
            filename = f"{ticker}_30min_data_{days}days.csv"
            stock_data.to_csv(filename, index=False)
            print(f"💾 데이터가 '{filename}'에 저장되었습니다.")
        
        return stock_data
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print(f"오류 상세: {type(e).__name__}")
        return None

def get_max_period_data(ticker, save_to_csv=True):
    """
    yfinance 제약사항에 맞춰 가능한 최대 기간의 데이터를 수집
    - 1시간: 730일 (약 2년)
    - 30분: 60일
    - 15분: 60일
    - 5분: 60일
    - 1분: 7일
    """
    
    print(f"🚀 {ticker} 최대 기간 데이터 수집...")
    
    intervals_and_periods = [
        ('1h', 730, '2년'),
        ('30m', 60, '60일'),
        ('15m', 60, '60일'),
        ('5m', 60, '60일'),
        ('1m', 7, '7일')
    ]
    
    all_data = {}
    
    for interval, max_days, description in intervals_and_periods:
        try:
            print(f"\n📊 {interval} 간격 데이터 수집 중... (최대 {description})")
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=max_days)
            
            data = yf.download(
                ticker,
                start=start_date.strftime('%Y-%m-%d'),
                end=end_date.strftime('%Y-%m-%d'),
                interval=interval,
                prepost=True,  # 시장 외 시간 데이터 포함
                progress=False
            )
            
            if not data.empty:
                data = data.reset_index()
                all_data[interval] = data
                print(f"✅ {interval} 데이터: {len(data):,}개 포인트")
                
                if save_to_csv:
                    filename = f"{ticker}_{interval}_data_{max_days}days.csv"
                    data.to_csv(filename, index=False)
                    print(f"💾 저장: {filename}")
            else:
                print(f"❌ {interval} 데이터 없음")
                
        except Exception as e:
            print(f"❌ {interval} 오류: {e}")
    
    return all_data

def get_longer_period_with_daily(ticker, days=365, save_to_csv=True):
    """
    1년 데이터가 필요한 경우 일별 데이터로 수집
    """
    
    try:
        print(f"📊 {ticker} 일별 데이터 수집 중... ({days}일)")
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        # 일별 데이터는 제약이 거의 없음
        daily_data = yf.download(
            ticker,
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d'),
            interval='1d',
            prepost=True,  # 시장 외 시간 데이터 포함
            progress=False
        )
        
        if daily_data.empty:
            print(f"❌ {ticker} 일별 데이터를 찾을 수 없습니다.")
            return None
        
        daily_data = daily_data.reset_index()
        
        print(f"✅ 일별 데이터 수집 완료!")
        print(f"총 데이터 포인트: {len(daily_data):,}개")
        print(f"데이터 기간: {daily_data['Date'].min()} ~ {daily_data['Date'].max()}")
        
        # 기술적 지표 추가
        daily_data = add_technical_features_daily(daily_data)
        
        if save_to_csv:
            filename = f"{ticker}_daily_data_{days}days.csv"
            daily_data.to_csv(filename, index=False)
            print(f"💾 데이터가 '{filename}'에 저장되었습니다.")
        
        return daily_data
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

def add_technical_features_daily(df):
    """일별 데이터용 기술적 지표 추가"""
    
    print("🔧 기술적 지표 계산 중...")
    
    # 수익률 계산
    df['Returns'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 이동평균
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # 지수이동평균
    df['EMA_12'] = df['Close'].ewm(span=12).mean()
    df['EMA_26'] = df['Close'].ewm(span=26).mean()
    
    # MACD
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # RSI
    df['RSI'] = calculate_rsi(df['Close'])
    
    # 볼린저 밴드
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    bb_std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # 변동성
    df['Volatility_5'] = df['Returns'].rolling(window=5).std()
    df['Volatility_10'] = df['Returns'].rolling(window=10).std()
    df['Volatility_20'] = df['Returns'].rolling(window=20).std()
    
    # 가격 변화
    df['Price_Change'] = df['Close'] - df['Open']
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # High-Low 스프레드
    df['HL_Spread'] = df['High'] - df['Low']
    df['HL_Spread_Pct'] = (df['High'] - df['Low']) / df['Close'] * 100
    
    # 시간 특성 (일별 데이터용)
    df['DayOfWeek'] = pd.to_datetime(df['Date']).dt.dayofweek
    df['Month'] = pd.to_datetime(df['Date']).dt.month
    df['Quarter'] = pd.to_datetime(df['Date']).dt.quarter
    df['DayOfMonth'] = pd.to_datetime(df['Date']).dt.day
    df['WeekOfYear'] = pd.to_datetime(df['Date']).dt.isocalendar().week
    
    # 거래일 특성
    df['Is_Monday'] = (df['DayOfWeek'] == 0).astype(int)
    df['Is_Friday'] = (df['DayOfWeek'] == 4).astype(int)
    df['Is_MonthEnd'] = pd.to_datetime(df['Date']).dt.is_month_end.astype(int)
    df['Is_MonthStart'] = pd.to_datetime(df['Date']).dt.is_month_start.astype(int)
    
    print(f"✅ 기술적 지표 추가 완료! 총 컬럼 수: {len(df.columns)}개")
    
    return df

def add_technical_features(df):
    """기술적 지표 추가"""
    
    print("🔧 기술적 지표 계산 중...")
    
    # 수익률 계산
    df['Returns'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 이동평균
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # 지수이동평균
    df['EMA_12'] = df['Close'].ewm(span=12).mean()
    df['EMA_26'] = df['Close'].ewm(span=26).mean()
    
    # MACD
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # RSI
    df['RSI'] = calculate_rsi(df['Close'])
    
    # 볼린저 밴드
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    bb_std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # 변동성
    df['Volatility_10'] = df['Returns'].rolling(window=10).std()
    df['Volatility_20'] = df['Returns'].rolling(window=20).std()
    
    # 가격 변화
    df['Price_Change'] = df['Close'] - df['Open']
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # High-Low 스프레드
    df['HL_Spread'] = df['High'] - df['Low']
    df['HL_Spread_Pct'] = (df['High'] - df['Low']) / df['Close'] * 100
    
    # 시간 특성
    df['Hour'] = df['Datetime'].dt.hour
    df['DayOfWeek'] = df['Datetime'].dt.dayofweek
    df['Month'] = df['Datetime'].dt.month
    df['Quarter'] = df['Datetime'].dt.quarter
    
    # 거래시간 여부 (미국 주식시장: 9:30-16:00 EST, 24시간 포함으로 확대)
    df['Is_Trading_Hours'] = ((df['Hour'] >= 9) & (df['Hour'] <= 16)).astype(int)  # 정규 거래시간
    df['Is_Market_Open'] = ((df['Hour'] >= 9) & (df['Hour'] < 16)).astype(int)     # 시장 개장시간
    df['Is_Premarket'] = ((df['Hour'] >= 4) & (df['Hour'] < 9)).astype(int)       # 프리마켓 (4:00-9:30)
    df['Is_Aftermarket'] = ((df['Hour'] >= 16) & (df['Hour'] <= 20)).astype(int)  # 애프터마켓 (16:00-20:00)
    df['Is_Extended_Hours'] = (df['Is_Premarket'] | df['Is_Aftermarket']).astype(int)  # 연장거래시간
    
    print(f"✅ 기술적 지표 추가 완료! 총 컬럼 수: {len(df.columns)}개")
    
    return df

def calculate_rsi(prices, window=14):
    """RSI (Relative Strength Index) 계산"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def adjust_time_to_hour(df):
    """시간을 정시로 조정하는 함수 (예: 13:30 -> 13:00)"""
    
    print("🕐 시간을 정시로 조정 중...")
    
    # Datetime 컬럼이 있는지 확인
    if 'Datetime' in df.columns:
        # 시간을 정시로 조정 (분, 초를 0으로 설정)
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        df['Datetime'] = df['Datetime'].dt.floor('H')  # 시간 단위로 내림
        
        print(f"✅ 시간 조정 완료: {df['Datetime'].min()} ~ {df['Datetime'].max()}")
        
        # 중복된 시간이 있는 경우 마지막 값 유지
        df = df.drop_duplicates(subset=['Datetime'], keep='last')
        print(f"중복 제거 후 데이터 포인트: {len(df):,}개")
        
    return df

def get_multiple_tickers_hourly(tickers, days=365, save_individual=True, save_combined=True):
    """여러 티커의 1시간 간격 데이터를 한번에 수집"""
    
    print(f"🚀 {len(tickers)}개 티커 1시간 간격 데이터 수집 시작...")
    print(f"티커 목록: {', '.join(tickers)}")
    print(f"수집 기간: 최근 {days}일")
    print("=" * 60)
    
    all_data = {}
    
    for i, ticker in enumerate(tickers, 1):
        print(f"\n[{i}/{len(tickers)}] {ticker} 처리 중...")
        
        data = get_hourly_stock_data(ticker, days=days, save_to_csv=save_individual)
        
        if data is not None:
            all_data[ticker] = data
            print(f"✅ {ticker} 완료")
        else:
            print(f"❌ {ticker} 실패")
        
        print("-" * 40)
    
    # 통합 데이터 저장
    if save_combined and all_data:
        print(f"\n💾 통합 데이터 저장 중...")
        
        # 각 티커별로 컬럼에 티커명 추가
        combined_data = pd.DataFrame()
        
        for ticker, data in all_data.items():
            ticker_data = data.copy()
            ticker_data['Ticker'] = ticker
            combined_data = pd.concat([combined_data, ticker_data], ignore_index=True)
        
        combined_filename = f"multiple_stocks_1hour_data_{days}days.csv"
        combined_data.to_csv(combined_filename, index=False)
        print(f"✅ 통합 데이터가 '{combined_filename}'에 저장되었습니다.")
        print(f"총 데이터 포인트: {len(combined_data):,}개")
    
    return all_data

def get_multiple_tickers(tickers, days=60, save_individual=True, save_combined=True):
    """여러 티커의 30분 간격 데이터를 한번에 수집"""
    
    print(f"🚀 {len(tickers)}개 티커 30분 간격 데이터 수집 시작...")
    print(f"티커 목록: {', '.join(tickers)}")
    print(f"수집 기간: 최근 {days}일")
    print("=" * 60)
    
    all_data = {}
    
    for i, ticker in enumerate(tickers, 1):
        print(f"\n[{i}/{len(tickers)}] {ticker} 처리 중...")
        
        data = get_30min_stock_data(ticker, days=days, save_to_csv=save_individual)
        
        if data is not None:
            all_data[ticker] = data
            print(f"✅ {ticker} 완료")
        else:
            print(f"❌ {ticker} 실패")
        
        print("-" * 40)
    
    # 통합 데이터 저장
    if save_combined and all_data:
        print(f"\n💾 통합 데이터 저장 중...")
        
        # 각 티커별로 컬럼에 티커명 추가
        combined_data = pd.DataFrame()
        
        for ticker, data in all_data.items():
            ticker_data = data.copy()
            ticker_data['Ticker'] = ticker
            combined_data = pd.concat([combined_data, ticker_data], ignore_index=True)
        
        combined_filename = f"multiple_stocks_30min_data_{days}days.csv"
        combined_data.to_csv(combined_filename, index=False)
        print(f"✅ 통합 데이터가 '{combined_filename}'에 저장되었습니다.")
        print(f"총 데이터 포인트: {len(combined_data):,}개")
    
    return all_data

def analyze_data_summary(data_dict):
    """수집된 데이터 요약 분석"""
    
    print("\n" + "=" * 60)
    print("📊 데이터 수집 요약")
    print("=" * 60)
    
    for ticker, data in data_dict.items():
        if data is not None:
            print(f"\n{ticker}:")
            print(f"  데이터 포인트: {len(data):,}개")
            print(f"  기간: {data['Datetime'].min().strftime('%Y-%m-%d %H:%M')} ~ {data['Datetime'].max().strftime('%Y-%m-%d %H:%M')}")
            print(f"  가격 범위: ${data['Low'].min():.2f} ~ ${data['High'].max():.2f}")
            print(f"  평균 거래량: {data['Volume'].mean():,.0f}")
            
            # 결측치 확인
            missing_count = data.isnull().sum().sum()
            if missing_count > 0:
                print(f"  ⚠️ 결측치: {missing_count}개")
            else:
                print(f"  ✅ 결측치 없음")

# 사용 예시
if __name__ == "__main__":
    
    print("🎯 yfinance 제약사항 안내:")
    print("- 1시간 간격: 최대 730일 (약 2년) ⭐ 추천!")
    print("- 30분 간격: 최대 60일")
    print("- 일별 간격: 제한 없음")
    print("=" * 60)
    
    # 1. 1시간 간격 데이터 (1년) - 메인 추천!
    print("\n🎯 1시간 간격 데이터 수집 (1년) - 추천!")
    aapl_1h = get_hourly_stock_data('AAPL', days=365)
    
    if aapl_1h is not None:
        print(f"\n📋 AAPL 1시간 데이터 미리보기:")
        print(aapl_1h[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']].head())
        
        # 데이터 양 분석
        trading_hours = aapl_1h[aapl_1h['Is_Trading_Hours'] == 1]
        print(f"\n📊 LSTM 학습용 데이터 분석:")
        print(f"전체 시간: {len(aapl_1h):,}개")
        print(f"거래시간만: {len(trading_hours):,}개")
        print(f"LSTM 시퀀스 길이 30 가정 시 학습 샘플: {len(trading_hours) - 30:,}개")
    
    print("\n" + "="*80)
    
    # 2. 여러 티커 1시간 데이터 (1년)
    print("\n🎯 여러 티커 1시간 데이터 수집 (1년)")
    tickers = ['AAPL', 'AMZN', 'TSLA', 'GOOGL', 'MSFT']
    
    all_stock_data = get_multiple_tickers_hourly(tickers, days=365)
    
    # 3. 요약 분석
    analyze_data_summary(all_stock_data)
    
    print("\n" + "="*80)
    
    # 4. 30분 간격 비교용 (60일)
    print("\n🎯 30분 간격 데이터 비교 (60일)")
    print("⚠️ 30분 간격은 최대 60일 제한이 있습니다.")
    
    # 현재 날짜 확인
    current_date = datetime.now()
    print(f"현재 날짜: {current_date.strftime('%Y-%m-%d')}")
    
    aapl_30m = get_30min_stock_data('AAPL', days=30)  # 30일로 줄여서 안전하게 테스트
    
    if aapl_30m is not None:
        print(f"\n📋 AAPL 30분 데이터 미리보기:")
        print(aapl_30m[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']].head())
    
    print("\n🎉 모든 데이터 수집 완료!")
    print("\n💡 권장사항:")
    print("✅ 1시간 간격 1년 데이터 - LSTM 학습에 최적!")
    print(f"   → 약 {365 * 6.5:.0f}개 거래시간 데이터 포인트")
    print("   → 충분한 데이터 양 + 적절한 시간 해상도")
    print("⚠️ 30분 간격은 60일 제한으로 데이터 부족")
    print("⚠️ 일별 데이터는 시간 해상도 부족")


# rapid api

In [ ]:
#!/usr/bin/env python3
"""
RapidAPI - twitter241 엔드포인트를 사용한 트윗 크롤러
"""

import requests
import json
import csv
import os
import time
import logging
from datetime import datetime
from typing import List, Dict, Optional, Any

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('rapidapi_crawler.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class RapidAPITweetCrawler:
    """
    RapidAPI의 twitter241 엔드포인트를 사용하여 트윗을 수집하고 CSV로 저장하는 크롤러.
    페이지네이션(cursor)을 처리하여 지정된 개수만큼 트윗을 수집합니다.
    """
    def __init__(self, api_key: str):
        """
        크롤러를 초기화합니다.
        
        Args:
            api_key: RapidAPI에서 발급받은 API 키
        """
        if not api_key:
            raise ValueError("API 키가 제공되지 않았습니다.")
            
        self.api_key = api_key
        self.base_url = "https://twitter241.p.rapidapi.com/user-tweets"
        self.headers = {
            "x-rapidapi-key": self.api_key,
            "x-rapidapi-host": "twitter241.p.rapidapi.com"
        }
        # count를 증가시켜 한번에 더 많은 트윗 요청 (최대 200까지 시도)
        self.count_per_request = 200
        
        # cursor 캐시 및 중복 방지
        self.used_cursors = set()

    def _parse_tweets_from_response(self, response_json: Dict[str, Any]) -> List[Dict[str, str]]:
        """
        API 응답 JSON에서 트윗 데이터를 파싱합니다.
        
        Args:
            response_json: API로부터 받은 JSON 응답
        
        Returns:
            추출된 트윗 데이터 리스트 ({'created_at': ..., 'full_text': ...})
        """
        tweets_data = []
        
        try:
            # 'instructions' 리스트에서 'TimelineAddEntries' 타입의 항목을 찾습니다.
            instructions = response_json.get('result', {}).get('timeline', {}).get('instructions', [])
            
            timeline_entries = []
            for instruction in instructions:
                if instruction.get('type') == 'TimelineAddEntries':
                    timeline_entries = instruction.get('entries', [])
                    break
            
            if not timeline_entries:
                logger.warning("응답에서 'entries'를 찾을 수 없습니다.")
                return []

            for entry in timeline_entries:
                # 'TimelineTweet' 타입의 콘텐츠만 처리
                item_content = entry.get('content', {}).get('itemContent', {})
                if item_content and item_content.get('itemType') == 'TimelineTweet':
                    tweet_results = item_content.get('tweet_results', {})
                    result = tweet_results.get('result', {})
                    
                    # legacy 필드에 실제 데이터가 있습니다.
                    legacy_data = result.get('legacy', {})
                    
                    if legacy_data:
                        created_at = legacy_data.get('created_at', 'N/A')
                        full_text = ""
                        
                        # 리트윗(RT)인 경우 원본 트윗의 full_text를 가져옵니다.
                        # 'retweeted_status_result' 키가 있는지 확인합니다.
                        if 'retweeted_status_result' in legacy_data:
                            # 원본 트윗의 legacy 데이터를 찾습니다.
                            original_tweet_legacy = legacy_data.get('retweeted_status_result', {}).get('result', {}).get('legacy', {})
                            full_text = original_tweet_legacy.get('full_text', '')
                        else:
                            # 일반 트윗은 기존 방식대로 full_text를 가져옵니다.
                            full_text = legacy_data.get('full_text', '')

                        # 줄바꿈 문자를 공백으로 변환하고 양 끝 공백 제거
                        full_text = full_text.replace('\n', ' ').strip()
                        
                        tweets_data.append({
                            'created_at': created_at,
                            'full_text': full_text
                        })
        except (AttributeError, KeyError, IndexError) as e:
            logger.error(f"트윗 데이터 파싱 중 오류 발생: {e}")
            logger.debug(f"오류 발생 지점의 JSON 구조: {json.dumps(response_json, indent=2, ensure_ascii=False)}")
            
        return tweets_data

    def _find_next_cursor(self, response_json: Dict[str, Any]) -> Optional[str]:
        """
        API 응답에서 다음 페이지를 위한 cursor 값을 찾습니다.
        개선된 cursor 파싱으로 더 많은 cursor 타입을 처리합니다.
        
        Args:
            response_json: API로부터 받은 JSON 응답
            
        Returns:
            다음 페이지 cursor 문자열 또는 None
        """
        try:
            instructions = response_json.get('result', {}).get('timeline', {}).get('instructions', [])
            
            # 모든 instruction 타입에서 cursor 찾기
            all_cursors = []
            
            for instruction in instructions:
                # TimelineAddEntries에서 cursor 찾기
                if instruction.get('type') == 'TimelineAddEntries':
                    entries = instruction.get('entries', [])
                    for entry in entries:
                        content = entry.get('content', {})
                        if content.get('entryType') == 'TimelineTimelineCursor':
                            cursor_value = content.get('value')
                            cursor_type = content.get('cursorType', '')
                            
                            if cursor_value and cursor_value not in self.used_cursors:
                                all_cursors.append({
                                    'value': cursor_value,
                                    'type': cursor_type,
                                    'priority': 1 if cursor_type == 'Bottom' else 2
                                })
                
                # TimelineReplaceEntry에서도 cursor 찾기
                elif instruction.get('type') == 'TimelineReplaceEntry':
                    entry = instruction.get('entry', {})
                    content = entry.get('content', {})
                    if content.get('entryType') == 'TimelineTimelineCursor':
                        cursor_value = content.get('value')
                        cursor_type = content.get('cursorType', '')
                        
                        if cursor_value and cursor_value not in self.used_cursors:
                            all_cursors.append({
                                'value': cursor_value,
                                'type': cursor_type,
                                'priority': 1 if cursor_type == 'Bottom' else 2
                            })
            
            # cursor를 우선순위에 따라 정렬 (Bottom이 우선)
            if all_cursors:
                all_cursors.sort(key=lambda x: x['priority'])
                selected_cursor = all_cursors[0]['value']
                self.used_cursors.add(selected_cursor)
                logger.debug(f"선택된 cursor: {selected_cursor[:50]}... (타입: {all_cursors[0]['type']})")
                return selected_cursor
                
        except (AttributeError, KeyError, IndexError) as e:
            logger.error(f"Cursor 파싱 중 오류 발생: {e}")
            
        return None

    def fetch_user_tweets(self, user_id: str, max_tweets: int = 1000):
        """
        특정 사용자의 트윗을 수집하여 CSV 파일로 저장합니다.
        개선된 페이지네이션으로 더 많은 트윗을 효율적으로 수집합니다.
        
        Args:
            user_id: 트윗을 수집할 사용자의 ID
            max_tweets: 수집할 최대 트윗 수
        """
        logger.info(f"사용자 ID {user_id}의 트윗 수집을 시작합니다. 목표: {max_tweets}개")
        logger.info(f"한 번의 요청당 {self.count_per_request}개 트윗 요청")
        
        all_tweets = []
        cursor = None
        request_count = 0
        max_requests = 100  # 무한 루프 방지
        consecutive_empty_responses = 0
        
        # cursor 캐시 초기화
        self.used_cursors.clear()
        
        while len(all_tweets) < max_tweets and request_count < max_requests:
            # count를 동적으로 조정 (남은 트윗 수에 따라)
            remaining_tweets = max_tweets - len(all_tweets)
            current_count = min(self.count_per_request, remaining_tweets)
            
            querystring = {
                "user": user_id,
                "count": str(current_count)
            }
            if cursor:
                querystring["cursor"] = cursor
            
            logger.info(f"API 요청 #{request_count + 1}: {len(all_tweets)} / {max_tweets} 수집됨. Count: {current_count}")
            
            try:
                response = requests.get(self.base_url, headers=self.headers, params=querystring, timeout=45)
                request_count += 1
                
                if response.status_code == 429:  # Rate limit
                    logger.warning("Rate limit에 도달했습니다. 60초 대기...")
                    time.sleep(60)
                    continue
                elif response.status_code != 200:
                    logger.error(f"API 에러: {response.status_code} - {response.text}")
                    if response.status_code >= 500:  # 서버 에러인 경우 재시도
                        logger.info("서버 에러로 인한 10초 후 재시도...")
                        time.sleep(10)
                        continue
                    else:
                        break
                    
                data = response.json()
                
                newly_fetched_tweets = self._parse_tweets_from_response(data)
                
                if not newly_fetched_tweets:
                    consecutive_empty_responses += 1
                    logger.warning(f"이번 응답에서 트윗을 찾을 수 없습니다. ({consecutive_empty_responses}/3)")
                    
                    if consecutive_empty_responses >= 3:
                        logger.info("연속 3회 빈 응답으로 수집을 종료합니다.")
                        break
                else:
                    consecutive_empty_responses = 0
                    logger.info(f"이번 요청에서 {len(newly_fetched_tweets)}개 트윗 수집")
                
                all_tweets.extend(newly_fetched_tweets)
                
                # 중복 제거 (created_at + full_text 기준)
                seen = set()
                unique_tweets = []
                for tweet in all_tweets:
                    tweet_key = (tweet['created_at'], tweet['full_text'])
                    if tweet_key not in seen:
                        seen.add(tweet_key)
                        unique_tweets.append(tweet)
                
                all_tweets = unique_tweets
                logger.info(f"중복 제거 후: {len(all_tweets)}개 트윗")
                
                # 다음 cursor 찾기
                next_cursor = self._find_next_cursor(data)
                if not next_cursor or next_cursor == cursor:
                    logger.info("더 이상 사용 가능한 cursor가 없습니다. 수집을 종료합니다.")
                    break
                
                cursor = next_cursor

                # API rate limit를 고려한 대기 시간 (요청 수에 따라 조정)
                if request_count % 10 == 0:  # 10번째마다 긴 대기
                    wait_time = 5
                else:
                    wait_time = 1
                    
                logger.debug(f"{wait_time}초 대기 중...")
                time.sleep(wait_time)

            except requests.exceptions.Timeout:
                logger.warning("요청 타임아웃. 5초 후 재시도...")
                time.sleep(5)
                continue
            except requests.exceptions.RequestException as e:
                logger.error(f"네트워크 오류 발생: {e}")
                time.sleep(10)
                continue
            except json.JSONDecodeError:
                logger.error("JSON 디코딩 오류. 응답이 올바른 JSON 형식이 아닙니다.")
                time.sleep(5)
                continue

        logger.info(f"총 {len(all_tweets)}개의 트윗을 {request_count}번의 요청으로 수집했습니다.")
        logger.info(f"평균 요청당 트윗 수: {len(all_tweets) / request_count if request_count > 0 else 0:.1f}개")
        
        if all_tweets:
            filename = f"user_{user_id}_tweets_ReTweet.csv"
            self._save_to_csv(all_tweets, filename)
            
    def _save_to_csv(self, tweets_list: List[Dict[str, str]], filename: str):
        """
        수집된 트윗 데이터를 CSV 파일로 저장합니다.
        
        Args:
            tweets_list: 저장할 트윗 데이터 리스트
            filename: 저장할 파일 이름
        """
        try:
            with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
                # 'utf-8-sig'는 Excel에서 한글이 깨지지 않도록 BOM을 추가합니다.
                writer = csv.DictWriter(f, fieldnames=['created_at', 'full_text'])
                writer.writeheader()
                writer.writerows(tweets_list)
            logger.info(f"CSV 파일 저장 완료: {filename}")
        except IOError as e:
            logger.error(f"파일 저장 중 오류 발생: {e}")

def main():
    """
    스크립트 실행을 위한 메인 함수
    """
    print("=" * 70)
    print("  RapidAPI(twitter241) 기반 트윗 크롤러 (개선된 버전)")
    print("=" * 70)
    
    # --- 설정 ---
    # 보안을 위해 API 키는 환경 변수에서 가져오는 것을 권장합니다.
    # 예: api_key = os.getenv("RAPIDAPI_KEY")
    API_KEY = "5fac920861msh988e449f8d91b60p10459bjsnba691d3d2d81" # 사용자 요청에 따라 하드코딩
    USER_ID = "86437069"
    # @WhiteHouse 1879644163769335808
    # @SecScottBessent 1889019333960998912
    # @JDVance 1542228578
    # @marcorubio 15745368
    # @elonmusk 44196397
    MAX_TWEETS = 1000
    
    if not API_KEY:
        print("[ERROR] API 키가 설정되지 않았습니다. 스크립트를 종료합니다.")
        return
        
    print(f"대상 사용자 ID: {USER_ID}")
    print(f"수집 목표 트윗 수: {MAX_TWEETS}")
    print("-" * 70)
    
    crawler = RapidAPITweetCrawler(api_key=API_KEY)
    crawler.fetch_user_tweets(user_id=USER_ID, max_tweets=MAX_TWEETS)
    
    print("=" * 70)
    print("크롤링 작업이 완료되었습니다.")
    print(f"결과는 user_{USER_ID}_tweets_ReTweet.csv 파일에 저장되었습니다.")
    print("=" * 70)


if __name__ == "__main__":
    main() 

# LSTM + DNN 코드

In [12]:
import pandas as pd

# 파일 경로
stock_path = "./AAPL_1hour_data_365days.csv"
news_path = "./apple_finbert_finnhub.csv"

# 데이터 불러오기
stock_df = pd.read_csv(stock_path, parse_dates=["Datetime"])
news_df = pd.read_csv(news_path, parse_dates=["pubDate"])

# 타임존 제거
stock_df["Datetime"] = stock_df["Datetime"].dt.tz_localize(None)
news_df["pubDate"] = news_df["pubDate"].dt.tz_localize(None)

# 정렬
stock_df = stock_df.sort_values("Datetime").reset_index(drop=True)

# 제외할 열
exclude_cols = ['Is_Trading_Hours', 'Is_Market_Open', 'Is_Premarket', 'Is_Aftermarket', 'Is_Extended_Hours']
stock_df = stock_df.drop(columns=[col for col in exclude_cols if col in stock_df.columns])

# 병합 결과
rows = []

for _, news_row in news_df.iterrows():
    news_time = news_row['pubDate']

    # 뉴스 이후 가장 가까운 주가
    future_stock = stock_df[stock_df['Datetime'] > news_time].head(1)
    if future_stock.empty:
        continue

    target_row = future_stock.iloc[0]
    target_time = target_row['Datetime']
    target_close = target_row['Close']

    # 과거 3개 주가
    past_rows = stock_df[stock_df['Datetime'] < target_time].tail(3)
    if len(past_rows) < 3:
        continue

    past_last_close = past_rows.iloc[-1]['Close']

    # 상승률
    return_pct = (target_close - past_last_close) / past_last_close * 100
    label = 1 if return_pct >= 0.4 else (-1 if return_pct <= -0.4 else 0)

    # 병합 row 생성
    row = {
        "news_id": news_row['id'],
        "news_time": news_time,
        "target_close": target_close,
        "target_return_pct": return_pct,
        "target_multi_raw": label,
        "finbert_positive": news_row['finbert_positive'],
        "finbert_neutral": news_row['finbert_neutral'],
        "finbert_negative": news_row['finbert_negative'],
    }

    # 과거 3개 flatten
    for i, (_, stock_row) in enumerate(past_rows.iterrows(), 1):
        for col in stock_df.columns:
            if col == "Datetime":
                continue
            row[f"x{i}_{col}"] = stock_row[col]

    rows.append(row)

# 최종 DataFrame
merged_df = pd.DataFrame(rows)

# 클래스 0/1/2로 매핑 (XGBoost용)
label_map = {-1: 0, 0: 1, 1: 2}
merged_df["target_multi"] = merged_df["target_multi_raw"].map(label_map)

# 저장
merged_df.to_csv("news_stock_classification.csv", index=False)
print("병합 완료: news_stock_classification.csv 저장됨")


병합 완료: news_stock_classification.csv 저장됨


In [9]:

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. 데이터 불러오기
df = pd.read_csv("news_stock_classification.csv", parse_dates=["news_time"])

# 2. Feature 및 Label 준비
feature_cols = [col for col in df.columns if col.startswith("x") or col.startswith("finbert_")]
X = df[feature_cols].fillna(0)
y = df["target_multi"]

# 3. 시계열 데이터 3-step 생성 (x1_, x2_, x3_)
X_seq = []
for i in range(len(X)):
    X_seq.append([
        X.iloc[i][[col for col in X.columns if col.startswith("x1_")]].values,
        X.iloc[i][[col for col in X.columns if col.startswith("x2_")]].values,
        X.iloc[i][[col for col in X.columns if col.startswith("x3_")]].values
    ])
X_seq = np.array(X_seq)

# 4. FinBERT 피처 추가 (Broadcast across time steps)
finbert_feats = X[[c for c in X.columns if c.startswith("finbert_")]].values
finbert_feats = np.repeat(finbert_feats[:, np.newaxis, :], 3, axis=1)
X_seq = np.concatenate([X_seq, finbert_feats], axis=-1)

# 5. 정규화
n_samples, time_steps, n_features = X_seq.shape
X_reshaped = X_seq.reshape(-1, n_features)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reshaped)
X_seq = X_scaled.reshape(n_samples, time_steps, n_features)

# 6. Tensor로 변환
X_tensor = torch.tensor(X_seq, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

# 7. Train/Test 분리
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)
train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_dl = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

# 8. LSTM 모델 정의
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1])

# 9. 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(input_dim=n_features).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 10. 학습 루프
for epoch in range(50):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

Epoch 1 | Loss: 165.6778
Epoch 2 | Loss: 139.9572
Epoch 3 | Loss: 124.0224
Epoch 4 | Loss: 111.9599
Epoch 5 | Loss: 102.0316
Epoch 6 | Loss: 91.2283
Epoch 7 | Loss: 82.4791
Epoch 8 | Loss: 74.9245
Epoch 9 | Loss: 68.3347
Epoch 10 | Loss: 62.1140
Epoch 11 | Loss: 56.4879
Epoch 12 | Loss: 51.9471
Epoch 13 | Loss: 47.4376
Epoch 14 | Loss: 43.8949
Epoch 15 | Loss: 39.6787
Epoch 16 | Loss: 36.4289
Epoch 17 | Loss: 33.2387
Epoch 18 | Loss: 30.9987
Epoch 19 | Loss: 28.0263
Epoch 20 | Loss: 25.6803
Epoch 21 | Loss: 23.1786
Epoch 22 | Loss: 21.3123
Epoch 23 | Loss: 19.6293
Epoch 24 | Loss: 17.5688
Epoch 25 | Loss: 15.6692
Epoch 26 | Loss: 14.3345
Epoch 27 | Loss: 12.6775
Epoch 28 | Loss: 11.6631
Epoch 29 | Loss: 10.7014
Epoch 30 | Loss: 9.4993
Epoch 31 | Loss: 8.4388
Epoch 32 | Loss: 7.4168
Epoch 33 | Loss: 7.0659
Epoch 34 | Loss: 5.9982
Epoch 35 | Loss: 5.2894
Epoch 36 | Loss: 4.8408
Epoch 37 | Loss: 4.3750
Epoch 38 | Loss: 3.9858
Epoch 39 | Loss: 4.1333
Epoch 40 | Loss: 3.2084
Epoch 41 | Loss

In [ ]:

# 11. 평가
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for xb, yb in test_dl:
        xb = xb.to(device)
        preds = model(xb).argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(yb.numpy())

print("\n Accuracy:", accuracy_score(all_labels, all_preds))
print("\n Classification Report:\n", classification_report(all_labels, all_preds))


 Accuracy: 0.5917297612114153

 Classification Report:
               precision    recall  f1-score   support

           0       0.09      0.13      0.11       246
           1       0.77      0.78      0.77      1224
           2       0.25      0.11      0.15       247

    accuracy                           0.59      1717
   macro avg       0.37      0.34      0.34      1717
weighted avg       0.59      0.59      0.59      1717


 Confusion Matrix:
 [[ 32 161  53]
 [237 957  30]
 [ 90 130  27]]
